In [29]:
import networkTransit as nt

In [30]:
data = pd.read_csv('SperoniOSRMdata.csv')

In [31]:
data

,trip_id,origin_lat,origin_lon,dest_lat,dest_lon,dur_actual,dist_pred,sched_end,date_only,sched_end_time_only,day_of_week,dep_origin,arr_at_dest
0,1248167,33.91,-118.35,33.92,-118.35,7,1,1/28/19 7:38,1/28/19 0:00,7:38:17,Monday,1/28/19 7:29,1/28/19 7:36
1,1108571,34.05,-118.24,34.05,-118.26,12,1,12/6/18 7:28,12/6/18 0:00,7:28:48,Thursday,12/6/18 7:32,12/6/18 7:44
2,1229641,33.94,-118.02,33.97,-118.08,16,4,1/25/19 6:32,1/25/19 0:00,6:32:36,Friday,1/25/19 6:23,1/25/19 6:39
3,1022430,34.00,-118.46,34.03,-118.46,13,4,10/26/18 7:29,10/26/18 0:00,7:29:45,Friday,10/26/18 7:19,10/26/18 7:33
4,1327145,33.87,-118.25,33.73,-118.30,30,12,2/19/19 7:23,2/19/19 0:00,7:23:17,Tuesday,2/19/19 7:00,2/19/19 7:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32791,1304850,33.87,-118.09,33.89,-118.23,31,10,5/15/19 7:40,5/15/19 0:00,7:40:00,Wednesday,5/15/19 7:13,5/15/19 7:44
32792,1146061,34.03,-118.35,33.96,-118.29,23,11,3/8/19 7:11,3/8/19 0:00,7:11:22,Friday,3/8/19 6:51,3/8/19 7:14
32793,1438088,33.96,-118.26,34.04,-117.75,52,40,4/12/19 7:45,4/12/19 0:00,7:45:00,Friday,4/12/19 6:45,4/12/19 7:37
32794,1473101,34.06,-118.27,34.16,-118.09,24,16,4/22/19 7:20,4/22/19 0:00,7:20:00,Monday,4/22/19 6:58,4/22/19 7:22


In [1]:
import requests
import pandas as pd
import multiprocessing as mp
import datetime as dt

In [2]:
data = pd.read_csv('SperoniOSRMdata.csv')
keys = data.keys()
data = pd.DataFrame(data,columns=[keys[0],keys[1],keys[2],keys[3],keys[4],keys[9],keys[8]])
keys = data.keys()

In [28]:
MAXW = 805

'{0:1.1f}'.format(MAXW/1609)

'0.5'

In [23]:
cores = mp.cpu_count()
rows = 4
group = rows // cores


mpCount = [(group*i,group*(i+1)) if i < cores-1
           else (group*i,rows)
           for i in range(cores)]

print(mpCount)

[(0, 1), (1, 2), (2, 3), (3, 4)]


In [4]:
def mp_transitTime(index):
    cols = ['Trip ID','Duration (min)','Walking Time (min)','Transit Time (min)',
            'Walking Distance (Mi)','Transfers','Message']
    responses = {cols[0]:[],cols[1]:[],cols[2]:[],
                 cols[3]:[],cols[4]:[],cols[5]:[],cols[6]:[]}
    worker = mp.current_process()
    wid = worker.name

    for i in range(index[0],index[1]):
        # collecting values
        vals = []
        for j in range(0,len(keys)):
            vals.append(data[keys[j]][i])
        vals[6] = vals[6][0:-5]

        # url for calling the server
        localhost = 'http://127.0.0.1:8080/otp/routers/default/'
        url = localhost + 'plan?'
        url += 'fromPlace={0},{1}'.format(vals[1],vals[2])
        url += '&toPlace={0},{1}'.format(vals[3],vals[4])
        url += '&time={0}'.format(vals[5])
        url += '&date={0}'.format(vals[6])
        url += '&mode=TRANSIT,WALK'
        # 805, 3220, 8047
        url += '&maxWalkDistance=805'
        url += '&arriveBy=true'
        url += '&optimize=QUICK'
        response = requests.get(url).json()

        responses[cols[0]].append(vals[0])
        if 'plan' in response:
            r = response['plan']['itineraries']

            responses[cols[1]].append(r[0]['duration']/60)
            responses[cols[2]].append(r[0]['walkTime']/60)
            responses[cols[3]].append(r[0]['transitTime']/60)
            responses[cols[4]].append(r[0]['walkDistance']/1609.34)
            responses[cols[5]].append(r[0]['transfers'])
            responses[cols[6]].append('Successful Run')
        else:
            responses[cols[1]].append(None)
            responses[cols[2]].append(None)
            responses[cols[3]].append(None)
            responses[cols[4]].append(None)
            responses[cols[5]].append(None)
            responses[cols[6]].append(response['error']['msg'][0:14])
            
    now = dt.datetime.now().strftime("%H%M")
    print('index',index,'done processing', now, wid)
    return responses

In [5]:
pool = mp.Pool(cores)

now = dt.datetime.now().strftime("%H%M%S")
print('Starting processing at', now)

results = pool.map(mp_transitTime,mpCount)

now = dt.datetime.now().strftime("%H%M%S")
print('Finished processing at', now)

df = pd.concat([pd.DataFrame(d) for d in results],ignore_index=True)
    
pool.close()

print(df.shape)
df.head()

Starting processing at 154007
index (0, 8199) done processing 1607 ForkPoolWorker-1
index (16398, 24597) done processing 1608 ForkPoolWorker-3
index (24597, 32796) done processing 1608 ForkPoolWorker-4
index (8199, 16398) done processing 1611 ForkPoolWorker-2
Finished processing at 161127
(32796, 7)


,Trip ID,Duration (min),Walking Time (min),Transit Time (min),Walking Distance (m),Transfers,Message
0,1248167,16.966667,16.966667,0.000000,1315.121301,0.0,Successful Run
1,1108571,22.150000,18.500000,3.616667,1400.756331,0.0,Successful Run
2,1229641,NaN,NaN,NaN,NaN,NaN,No trip found.
3,1022430,NaN,NaN,NaN,NaN,NaN,No trip found.
4,1327145,NaN,NaN,NaN,NaN,NaN,No trip found.


In [6]:
now = dt.datetime.now().strftime("%Y%m%d-%H%M")
df.to_csv(now + 'transitTimesTwoMiles.csv')

In [13]:
df['Message'].value_counts()

No trip found.    25344
Successful Run     7417
Origin is with       35
Name: Message, dtype: int64